# Mount Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Framework

In [4]:
################################################################################
# Libraries
################################################################################
#!pip install keras-preprocessing
import sys
from google.colab import output
import cv2
import numpy as np
import PIL
import io
import numpy
from scipy.ndimage.filters import gaussian_filter
from IPython.display import Image
from IPython.display import Image, display
from google.colab.patches import cv2_imshow
from matplotlib import pyplot as plt
import csv
import time
from PIL import Image
import random
from operator import itemgetter
import matplotlib
import matplotlib.pyplot
import os
import matplotlib.pyplot as plt
from argparse import ArgumentParser
from collections import OrderedDict
np.set_printoptions(threshold=sys.maxsize)
from tensorflow.keras.models import Sequential,load_model
from sklearn.metrics import classification_report, confusion_matrix
import warnings
warnings.filterwarnings('ignore')
from google.colab.patches import cv2_imshow
import tensorflow as tf
import pandas as pd
from keras_preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.preprocessing import image
from os import listdir
from os.path import isfile, join
import seaborn as sns
from tqdm import tqdm
from time import sleep
from IPython.display import clear_output
%matplotlib inline
import matplotlib.pyplot as plt
from shapely.geometry import MultiPoint, Point, Polygon
from shapely.ops import polygonize,unary_union
from shapely.geometry import LineString, MultiPolygon, MultiPoint, Point
from scipy.spatial import Voronoi
from scipy import spatial
from scipy.spatial import distance
import pylab
from scipy.spatial import Voronoi, voronoi_plot_2d
import matplotlib.lines as lines
import math
import matplotlib.patches as mpatches

# Access VCNN4PuDe Folder
%cd /content/drive/My Drive/VCNN4PuDe/

################################################################################
# Loading the trained model with its input image dim and threshold
################################################################################
# Loading the trained model
CNN_path="models/EfficientNetV1B0.h5"
CNN_model= load_model(CNN_path)
CNN_model.load_weights(CNN_path)
# Input image dim
img_rows, img_cols = 224,224
#Threshold value for the loaded classifier
threshold=0.3


################################################################################
# Dummy Points Generation and Neighbours Extraction
################################################################################
#Finding the distance between two 2D points
def find_distance(x1 , y1 , x2 , y2):
    return round( math.sqrt(math.pow(x2 - x1, 2) +
                math.pow(y2 - y1, 2) * 1.0),2)

#Kindly note that the author of voronoi_finite_polygons_2d function is https://gist.github.com/pv/8036995
#Reconstructing infinite Voronoi regions in a 2D diagram to finite regions
def voronoi_finite_polygons_2d(vor, radius=None):

    if vor.points.shape[1] != 2:
        raise ValueError("Requires 2D input")

    new_regions = []
    new_vertices = vor.vertices.tolist()

    center = vor.points.mean(axis=0)
    if radius is None:
        radius = vor.points.ptp().max()*2

    all_ridges = {}
    for (p1, p2), (v1, v2) in zip(vor.ridge_points, vor.ridge_vertices):
        all_ridges.setdefault(p1, []).append((p2, v1, v2))
        all_ridges.setdefault(p2, []).append((p1, v1, v2))

    # Reconstruct infinite regions
    for p1, region in enumerate(vor.point_region):
        vertices = vor.regions[region]

        if all(v >= 0 for v in vertices):
            # finite region

            new_regions.append(vertices)
            continue
        # reconstruct a non-finite region
        ridges = all_ridges[p1]
        new_region = [v for v in vertices if v >= 0]
        for p2, v1, v2 in ridges:
            if v2 < 0:
                v1, v2 = v2, v1
            if v1 >= 0:
                # finite ridge: already in the region
                continue
            # Compute the missing endpoint of an infinite ridge
            t = vor.points[p2] - vor.points[p1] # tangent
            t /= np.linalg.norm(t)
            n = np.array([-t[1], t[0]])  # normal

            midpoint = vor.points[[p1, p2]].mean(axis=0)
            direction = np.sign(np.dot(midpoint - center, n)) * n
            far_point = vor.vertices[v2] + direction * radius

            new_region.append(len(new_vertices))
            new_vertices.append(far_point.tolist())

        vs = np.asarray([new_vertices[v] for v in new_region])
        c = vs.mean(axis=0)
        angles = np.arctan2(vs[:,1] - c[1], vs[:,0] - c[0])
        new_region = np.array(new_region)[np.argsort(angles)]

        # finish
        new_regions.append(new_region.tolist())

    return new_regions, np.asarray(new_vertices)

 #------------------------------------------
#Finding neighbors of target_point_index within a given radius
def find_neighbors(new_vertices, target_point_index, radius, points):
    ne_regions=[]
    for ind in range(len(new_vertices)):
        # Find the number of common points between two regions
        a=np.array(new_vertices[ind])
        b=np.array(new_vertices[target_point_index])
        aset = set([tuple(x) for x in a])
        bset = set([tuple(x) for x in b])
        intersection = np.array([x for x in aset & bset])
        if len(intersection)==2 and find_distance(points[target_point_index,0],points[target_point_index,1], points[ind,0],points[ind,1])<=radius:
            ne_regions.append(ind)
    return (ne_regions)

#Updating the points of the border cells
def cropped_vor(points):
    vor = Voronoi(points)
    regions, vertices = voronoi_finite_polygons_2d(vor)
    pts = MultiPoint([Point(i) for i in points])
    mask = pts.convex_hull
    new_vertices = []
    #Crooped regions at the border
    for region in regions:
        polygon = vertices[region]
        shape = list(polygon.shape)
        shape[0] += 1
        p = Polygon(np.append(polygon, polygon[0]).reshape(*shape)).intersection(mask)
        poly = np.array(list(zip(p.boundary.coords.xy[0][:-1], p.boundary.coords.xy[1][:-1])))
        new_vertices.append(poly)
    return  new_vertices

def contain(x1,y1,x2,y2, x,y):
    if x>=x1 and x<=x2 and y>=y1 and y<=y2:
        return 1
    else:
        return 0

#Dummy points generation
def dummy_generation(inputFile):

  file = open(inputFile,encoding='utf-8-sig')
  data = np.loadtxt(file, delimiter=",")
  frames=np.unique(data[:,1])
  dummy_points=data

  ped_no=1000
  distance=220
  distance_for_point=160

  for fr in frames:
     current_fr=data[np.where((data[:,1]==fr) ) ]
     pedestrians=np.unique(current_fr[:,0])

     for row in pedestrians:
          ped=current_fr[np.where((current_fr[:,0]==int(row)) ) ]

          x=int(ped[0][2])
          y=int(ped[0][3])
          x1=x-distance
          y1=y-distance
          x2=x+distance
          y2=y+distance

          top_left_x1=x1
          top_left_y1=y1
          top_left_x2=x
          top_left_y2=y
          top_right_x1=x
          top_right_y1=y1
          top_right_x2=x2
          top_right_y2=y

          bottom_left_x1=x1
          bottom_left_y1=y
          bottom_left_x2=x
          bottom_left_y2=y2

          bottom_right_x1=x
          bottom_right_y1=y
          bottom_right_x2=x2
          bottom_right_y2=y2
          neighbours=current_fr[np.where((current_fr[:,0]!=int(row)) ) ]


          top_left_flag=0
          for ne in neighbours:

              if contain(top_left_x1,top_left_y1,top_left_x2,top_left_y2,ne[2],ne[3]):
                      top_left_flag=1


          top_right_flag=0
          for ne in neighbours:
              if contain(top_right_x1,top_right_y1,top_right_x2,top_right_y2,ne[2],ne[3]):
                      top_right_flag=1



          bottom_right_flag=0
          for ne in neighbours:
              if contain(bottom_right_x1,bottom_right_y1,bottom_right_x2,bottom_right_y2,ne[2],ne[3]):
                      bottom_right_flag=1


          bottom_left_flag=0
          for ne in neighbours:
              if contain(bottom_left_x1,bottom_left_y1,bottom_left_x2,bottom_left_y2,ne[2],ne[3]):
                      bottom_left_flag=1

          if top_left_flag==0:
            new_point=[top_left_x1+distance_for_point,top_left_y1+distance_for_point]
            rec=[ped_no,int(fr),int(new_point[0]),int(new_point[1])]
            dummy_points=np.append(dummy_points, [rec], axis=0)
            ped_no=ped_no+1


          if top_right_flag==0:
              new_point=[top_right_x2-distance_for_point,top_right_y1+distance_for_point]
              rec=[ped_no,int(fr),int(new_point[0]),int(new_point[1])]
              dummy_points=np.append(dummy_points, [rec], axis=0)
              ped_no=ped_no+1


          if bottom_right_flag==0:
              new_point=[bottom_right_x2-distance_for_point,bottom_right_y2-distance_for_point]
              rec=[ped_no,int(fr),int(new_point[0]),int(new_point[1])]
              dummy_points=np.append(dummy_points, [rec], axis=0)
              ped_no=ped_no+1


          if bottom_left_flag==0:
              new_point=[bottom_left_x1+distance_for_point,bottom_left_y2-distance_for_point]
              rec=[ped_no,int(fr),int(new_point[0]),int(new_point[1])]
              dummy_points=np.append(dummy_points, [rec], axis=0)
              ped_no=ped_no+1


  return dummy_points


#Find neighbours
# number_of_frames: Interval of keyframe extraction
def neighbours(trajectory,radius,number_of_frames,initial_frame):

    maxFrames=int(np.amax(trajectory[:,1]))
    #initial_frame=int(number_of_frames/2)

    ne_data=np.empty([1,5] )
    for fr in range(initial_frame,maxFrames,number_of_frames):
        if fr >=initial_frame:

            mid_fr=fr
            currentFrame=trajectory[np.where((trajectory[:,1] ==mid_fr)) ]
            points=currentFrame[:,2:4]
            if len(points)>1:
                pedId= currentFrame[:,0]
                vert=cropped_vor(points)

                for target_ind in range(len(points)):
                    #target_ped
                    target_ped=int(pedId[target_ind])

                    rec=[mid_fr,target_ped,target_ped, points[target_ind,0],points[target_ind,1]]
                    ne_data=np.append(ne_data, [rec], axis=0)
                    #Direct neighbours indices within a radius
                    neighbours_v=find_neighbors(vert,target_ind,radius,points)
                    if len(neighbours_v)>0:
                        for ne_ind in neighbours_v:
                            distance=find_distance(points[target_ind,0],points[target_ind,1],points[ne_ind,0],points[ne_ind,1] )
                            rec=[mid_fr,target_ped,int(pedId[ne_ind]), points[ne_ind,0],points[ne_ind,1] ]
                            ne_data=np.append(ne_data, [rec], axis=0)
    #Trajectory data with dummy points
    np.savetxt("newTrajectory.csv", ne_data, delimiter=',',fmt="%f")

################################################################################
#Local Region Extraction and pushing person annotaation
################################################################################
#Creating non-intersecting polygon, Thanks to the author of Create_random_polygon class
class Create_random_polygon:

    def __init__(self, array, min_rand_coord = None, max_rand_coord = None, points_num = None):
        self.array = array
        self.min_rand_coord = min_rand_coord
        self.max_rand_coord = max_rand_coord
        self.points_num = points_num

    def generate_random_points(self):
        random_coords_list = []
        for x in range(self.points_num):
            coords_tuple = (random.randint(self.min_rand_coord, self.max_rand_coord),
                            random.randint(self.min_rand_coord, self.max_rand_coord))
            random_coords_list.append(coords_tuple)
        self.array = random_coords_list
        return random_coords_list

    def close_line_to_polygon(self):
        a = self.array[0]
        b = self.array[len(self.array)-1]
        if a == b:
            pass
        else:
            self.array.append(a)

    def find_leftmost_point(self):
        leftmost_point = None
        leftmost_x = None
        for point in self.array:
            x = point[0]
            if leftmost_x == None or x < leftmost_x:
                leftmost_x = x
                leftmost_point = point
        return leftmost_point

    def find_rightmost_point(self):
        rightmost_point = None
        rightmost_x = None
        for point in self.array:
            x = point[0]
            if rightmost_x == None or x > rightmost_x:
                rightmost_x = x
                rightmost_point = point
        return rightmost_point

    def is_point_above_the_line(self, point, line_points):
        """return 1 if point is above the line
           return -1 if point is below the line
           return  0 if point is lays on the line"""
        px, py = point
        P1, P2 = line_points
        P1x, P1y = P1[0], P1[1]
        P2x, P2y = P2[0], P2[1]
        array = numpy.array([
            [P1x - px, P1y - py],
            [P2x - px, P2y - py],
            ])
        det = numpy.linalg.det(array)
        sign = numpy.sign(det)
        return sign

    def sort_array_into_A_B_C(self, line_points):
        [(x_lm, y_lm), (x_rm, y_rm)] = line_points
        A_array, B_array, C_array = [], [], []
        for point in self.array:
            x, y = point
            sing = self.is_point_above_the_line( (x, y), line_points)
            if sing == 0:
                C_array.append(point)
            elif sing == -1:
                A_array.append(point)
            elif sing == 1:
                B_array.append(point)
        return A_array, B_array, C_array

    def sort_and_merge_A_B_C_arrays(self, A_array, B_array, C_array):
        A_C_array = [*A_array, *C_array]
        A_C_array.sort(key=itemgetter(0))
        B_array.sort(key=itemgetter(0), reverse=True)
        merged_arrays = [*A_C_array, *B_array]
        self.array = merged_arrays

    def show_image(self, array, line_points, A_array, B_array, C_array):
        [(x_lm, y_lm), (x_rm, y_rm)] = line_points
        x = [x[0] for x in array]
        y = [y[1] for y in array]
        Ax = [x[0] for x in A_array]
        Ay = [y[1] for y in A_array]
        Bx = [x[0] for x in B_array]
        By = [y[1] for y in B_array]
        Cx = [x[0] for x in C_array]
        Cy = [y[1] for y in C_array]
        matplotlib.pyplot.plot(Ax, Ay, 'o', c='orange') # below the line
        matplotlib.pyplot.plot(Bx, By, 'o', c='blue') # above the line
        matplotlib.pyplot.plot(Cx, Cy, 'o', c='black') # on the line
        matplotlib.pyplot.plot(x_lm, y_lm, 'o', c='green') # leftmost point
        matplotlib.pyplot.plot(x_rm, y_rm, 'o', c='red') # rightmost point
        x_plot = matplotlib.pyplot.plot([x_lm, x_rm], [y_lm, y_rm], linestyle=':', color='black', linewidth=0.5) # polygon's division line
        x_plot = matplotlib.pyplot.plot(x, y, color='black', linewidth=1) # connect points by line in order of apperiance
        matplotlib.pyplot.show()

    def main(self, plot = False):
        'First output is random polygon coordinates array (other stuff for ploting)'
        #print(self.array)
        if self.array == None:
            if not all(
                [isinstance(min_rand_coord, int),
                 isinstance(max_rand_coord, int),
                 isinstance(points_num, int),]
                ):
                print('Error! Values must be "integer" type:', 'min_rand_coord =',min_rand_coord, ', max_rand_coord =',max_rand_coord, ', points_num =',points_num)
            else:
                self.array = self.generate_random_points()

        #print(self.array)
        x_lm, y_lm = self.find_leftmost_point()
        x_rm, y_rm = self.find_rightmost_point()
        line_points = [(x_lm, y_lm), (x_rm, y_rm)]

        A_array, B_array, C_array = self.sort_array_into_A_B_C(line_points)
        self.sort_and_merge_A_B_C_arrays(A_array, B_array, C_array)
        self.close_line_to_polygon()
        if plot:
            self.show_image(self.array, line_points, A_array, B_array, C_array)
        return self.array

# Local region classification
def classify(img,threshold):
    img= img[...,::-1]

    im_pil = Image.fromarray(img)
    im_resized = im_pil.resize((img_rows, img_cols),resample=0)

    img_array =img_to_array(  im_resized )
    image_array_expanded = np.expand_dims(img_array, axis = 0)

    group=CNN_model.predict(image_array_expanded,verbose = 0 )

    if(group>=threshold):
        return 1
    else:
        return 0
# find the required frame based on frame_id from the input video
def find_frame(video_name, frame_id):
  video = cv2.VideoCapture(video_name)
  video.set(cv2.CAP_PROP_POS_FRAMES, frame_id)
  ret, frame = video.read()
  return frame

# Extending 2D point
def extend(x1,y1,x,y):
    zx=x1-x
    zy=y1-y
    t=1.1
    x2=x+t*zx
    y2=y+t*zy
    return (x2,y2)

def delete (folder_path):
  for filename in os.listdir(folder_path):
      file_path = os.path.join(folder_path, filename)
      try:
          if os.path.isfile(file_path):
              os.remove(file_path)
          elif os.path.isdir(file_path):
              os.rmdir(file_path)
      except Exception as e:
          print(f"Error deleting {file_path}: {e}")

def annotation(video_name):

  data= open("newTrajectory.csv",encoding='utf-8-sig')
  da = np.loadtxt(data, delimiter=",")[1::,:]

  delete("annotated/")

  frame_ids=np.unique(da[:,0])
  for frame_id in tqdm(frame_ids,
              desc ="Annotataion"):

    img=find_frame(video_name,frame_id)

    pedestrians=da[np.where((da[:,0]==int(frame_id))) ]
    pedestrians=pedestrians[np.where((pedestrians[:,1]<1000)) ]
    ped=np.unique(pedestrians[:,1])

    for target in ped:
          ne=pedestrians[np.where((pedestrians[:,1]==int(target))) ]
          ne_without_target=ne[np.where((ne[:,2]!=int(target))) ]
          target_row=ne[np.where((ne[:,2]==int(target))) ]
          poly=[]
          for info in ne_without_target:
              if info[2]<1000:
                  extended_x,extended_y=extend(info[3],info[4],target_row[0][3],target_row[0][4])
              else:
                  extended_x,extended_y=info[3],info[4]
              poly.append([ extended_x,extended_y])
          pedId=int(ne[0,1])
          pedId=int(ne[0,1])
          min_rand_coord = 1
          max_rand_coord = 10000
          points_num = 30
          if len(poly)>0:
            crt = Create_random_polygon(poly, min_rand_coord, max_rand_coord, points_num)
            polygon_array = crt.main(plot = False)

            pts = np.array(polygon_array,np.int32)

            if len(pts)>2:
                mask = np.zeros(img.shape[0:2], dtype=np.uint8)
                #method 1 smooth region
                cv2.drawContours(mask, [pts], -1, (255, 255, 255), -1, cv2.LINE_AA)
                res = cv2.bitwise_and(img,img,mask = mask)
                rect = cv2.boundingRect(pts) # returns (x,y,w,h) of the rect
                cropped = res[rect[1]: rect[1] + rect[3], rect[0]: rect[0] + rect[2]]

                if pedId>0 and pedId<1000 and cropped.shape[0]>0:
                  if(classify(cropped,threshold)==1):
                    im = cv2.rectangle(img, (int(target_row[0][3])-25, int(target_row[0][4])-25),(int(target_row[0][3])+25, int(target_row[0][4])+25), (0,0,255), 3)
    #Store the annotated frames in the annotated folder
    cv2.imwrite("./annotated/"+str(frame_id)+".png",img)

def video_generation(fps):
  image_folder="annotated/"
  video_name = "annotated/annotatedVideo.mp4"
  images = [img for img in sorted(os.listdir(image_folder))
                if img.endswith(".jpg") or
                  img.endswith(".jpeg") or
                  img.endswith("png")]

  frameR = cv2.imread(os.path.join(image_folder, images[0]))

  width, height,c=frameR.shape

  fourcc = cv2.VideoWriter_fourcc(*'MP4V')
  #cv.VideoWriter_fourcc('M','J','P','G')
  video = cv2.VideoWriter(video_name,fourcc ,fps, (height, width))

  for image in images:
      video.write(cv2.imread(os.path.join(image_folder, image)))
  print("The video is created in the annotated directory under name annonatedVideo.mp4")


/content/drive/My Drive/VCNN4PuDe


#**Run the framework**
Main Inputs:
1. Trajectory file
2. Input Video
3. FPS for the generated annotated video

Output:
1. annonatedVideo.mp4

In [5]:
################################################################################
# Run the framework
# please determine the main inputs
################################################################################
#Main Inputs
trajectory_data="samples/trajectory-ROI.csv"
video_name="samples/50-undistorted.mp4"
fps=1
#Adding dummy point to trajectory data
dummy=dummy_generation(trajectory_data)
#Finding Direct neighbours and store in newTrajectory CSV file
neighbours(dummy,300,25,62)
# Annotation
annotation(video_name)
video_generation(fps)


Annotataion: 100%|██████████| 34/34 [01:59<00:00,  3.52s/it]


The video is created in the annotated directory under name annonatedVideo.mp4
